In [1]:
import pandas as pd

In [24]:
## Import data 
hours = pd.read_csv('PBJ_Daily_Nurse_Staffing_Q1_2024.csv',encoding='cp1252')
provider = pd.read_csv('NH_ProviderInfo_Aug2024.csv',encoding='cp1252')

## Format data
hours['WorkDate'] = pd.to_datetime(hours['WorkDate'],format='%Y%m%d')
hours.rename(columns={'ï»¿PROVNUM': 'Provider Number','PROVNAME':'Provider Name'}, inplace=True)
provider.rename(columns={'CMS Certification Number (CCN)':'Provider Number'},inplace = True)

C:\Users\17667\AppData\Local\Temp\ipykernel_1796\38420987.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  hours = pd.read_csv('PBJ_Daily_Nurse_Staffing_Q1_2024.csv',encoding='cp1252')


In [29]:
## Add column to calculate daily total hours and contract hours
df = hours
df['Daily_total_contract_hours'] = (df['Hrs_RNDON_ctr'] + df['Hrs_RNadmin_ctr'] + df['Hrs_RN_ctr'] +
                              df['Hrs_LPNadmin_ctr'] + df['Hrs_LPN_ctr'] + df['Hrs_CNA_ctr'] +
                              df['Hrs_NAtrn_ctr'] + df['Hrs_MedAide_ctr'])

df['Daily_total_hours'] = (df['Hrs_RNDON'] + df['Hrs_RNadmin'] + df['Hrs_RN'] +
                     df['Hrs_LPNadmin'] + df['Hrs_LPN'] + df['Hrs_CNA'] +
                     df['Hrs_NAtrn'] + df['Hrs_MedAide'])

## Group by facility name
df1 = df.groupby('Provider Number')[['Daily_total_hours','Daily_total_contract_hours']].mean()

# Calculate the contract hour percentage
df1['ctr_pct'] = df1['Daily_total_contract_hours']/df1['Daily_total_hours']*100
df1['ctr_pct'].sort_values(ascending=False).head(10)

# Merge df1 with provider information
data = pd.merge(df1,provider,on='Provider Number')

# Add three identification factors
# Contract hour percentile, the lowering meaning less contract percentage
data['ctr_%ile'] = round(data['ctr_pct'].rank(pct=True),1)*10

# Bed utilization rate
data['bed_utl_rate'] = data['Average Number of Residents per Day']/data['Number of Certified Beds']
data['bed_utl_%ile'] = round(data['bed_utl_rate'].rank(pct=True),1)*10

# Differentiate the size of hospitals
data['hospital_size'] = round(data['Number of Certified Beds'].rank(pct=True),1)*10


In [32]:
# Export data to excel
data.to_excel('2024Q1.xlsx')